In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import dotenv_values
import os

In [3]:
config = dotenv_values(".env")

In [41]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


In [42]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [43]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    # Function to check if action exists
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    # Function to call OpenAI
    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    # Function to take action
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [44]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=config["OPEN_AI_KEY"])  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [45]:
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

In [46]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_FssE4SWPsZlI26zV1dtBSuOV'}
Back to the model!


In [51]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FssE4SWPsZlI26zV1dtBSuOV', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b0eed02e-679c-4415-9086-5a1d82dcf859-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_FssE4SWPsZlI26zV1dtBSuOV'}]),
  ToolMessage(content='[{\'url\': \'https://world-weather.info/forecast/usa/san_francisco/june-2024/\', \'content\': \'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations -

In [52]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 68.5°F (20.3°C). The wind speed is 20.2 km/h coming from the west-northwest direction. The humidity is at 61%, and the visibility is 16.0 km.'

In [53]:
result['messages']

[HumanMessage(content='What is the weather in sf?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FssE4SWPsZlI26zV1dtBSuOV', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b0eed02e-679c-4415-9086-5a1d82dcf859-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_FssE4SWPsZlI26zV1dtBSuOV'}]),
 ToolMessage(content='[{\'url\': \'https://world-weather.info/forecast/usa/san_francisco/june-2024/\', \'content\': \'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations - World-Weather.

In [54]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_WsGHxlbDfeFuUemNSdPV7hYr'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_1yykou3Ho9eYYzxtUQXeiYMa'}
Back to the model!


In [55]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 68.5°F. The wind speed is 20.2 kph coming from the west-northwest direction. The humidity is at 61% with a visibility of 9.0 miles.\n\nIn Los Angeles, the weather is sunny with a temperature of 69.1°F. The wind speed is 20.2 kph coming from the west direction. The humidity is at 68% with clear skies and a visibility of 9.0 miles.'

In [57]:

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o", openai_api_key=config["OPEN_AI_KEY"])  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_yLZeEDdsAUjAi9F47nSPrVcs'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of state where 2024 Super Bowl winning team is headquartered'}, 'id': 'call_JJpXmiQq8kc6QP5QSTBp80wI'}
Back to the model!


In [60]:
print(result['messages'][-1].content)

### Who won the Super Bowl in 2024?

The Kansas City Chiefs won the Super Bowl in 2024. They defeated the San Francisco 49ers in an overtime victory with a score of 25-22.

### In what state is the winning team headquarters located?

The Kansas City Chiefs are headquartered in the state of Missouri.

### What is the GDP of that state?

According to the most recent data available from the Bureau of Economic Analysis, the Gross Domestic Product (GDP) of Missouri increased by an annual rate of 5.0 percent in the first quarter of 2024. For more specific and up-to-date figures, you may refer to the Bureau of Economic Analysis [GDP by State](https://www.bea.gov/data/gdp/gdp-state).
